In [1]:
import numpy as np
# from tqdm import tqdm

### Get Vocab

In [2]:
import pandas as pd
import re

from nltk.corpus import stopwords
from collections import Counter

In [3]:
# tags = ['economia','mundo','peru']

df1 = pd.read_csv('data/2021-02-03-news.csv')
df2 = pd.read_csv('data/2021-02-04-news.csv')
df3 = pd.read_csv('data/2021-02-05-news.csv')
df4 = pd.read_csv('data/2021-02-06-news.csv')
df5 = pd.read_csv('data/2021-02-07-news.csv')
df6 = pd.read_csv('data/2021-02-08-news.csv')
df7 = pd.read_csv('data/2021-02-09-news.csv')
df8 = pd.read_csv('data/2021-02-10-news.csv')
df9 = pd.read_csv('data/2021-02-11-news.csv')

dfx = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9])
dfx = dfx.iloc[:,1:]
print(dfx.shape)
dfx = dfx.drop_duplicates()
print(dfx.shape)
# dfx = dfx[dfx.topic.isin(tags)]
# print(dfx.shape)
dfx = dfx[~dfx.text.isnull()]
print(dfx.shape)
dfx = dfx.reset_index(drop=True)

(1483, 5)
(933, 5)
(928, 5)


In [12]:
dfx.to_csv('data/noticias.csv', index=False)

In [4]:
from bm25 import BM25
import unicodedata
import six
from nltk import word_tokenize
import io

In [5]:
def normalize_terms(terms):
    return [remove_diacritics(term).lower() for term in terms]

def remove_diacritics(text, encoding='utf8'):
    nfkd_form = unicodedata.normalize('NFKD', to_unicode(text, encoding))
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode(encoding)

def to_unicode(text, encoding='utf8'):
    if isinstance(text, six.text_type):
        return text
    return text.decode(encoding)

In [9]:
def do_bm25(query, df, best_n=3):

    query = normalize_terms(word_tokenize(query))

#     text_lines_total = []
#     for file_i in files:

#         fo = open(file_i, "r",encoding='utf-16')
#         text_lines = fo.readlines()

#         text_lines_total.extend(text_lines)

    text_lines_total = df.text.values

#     text_lines_tokens = [word_tokenize(tx_line) for tx_line in text_lines_total]

    text_lines_tokens = []
    for title in text_lines_total:
        
        title = re.sub(r'\W',' ', title) # matches any non-word character 
        title = re.sub(r'\s+',' ', title) # matches any whitespace character 
#         title = re.sub(r'\d+',' ', title) # matches a digit
        title = title.replace('  ', ' ')
        title = title.replace('   ', ' ')
#         title = title.replace('    ', ' ')
#         title = title.replace('     ', ' ')
        
        tx_line = word_tokenize(title)
        text_lines_tokens.append(tx_line)
    
    
#     print("amount of contexts: ", len(text_lines_tokens))

    news = [normalize_terms(sentence) for sentence in text_lines_tokens]
    bm25 = BM25(news)
    best_indexes = bm25.ranked(query, best_n)

    best_sentences = []

    for ind in best_indexes:
        sentence_found = text_lines_total[ind]
#         sentence_found = sentence_found.encode("ascii", "ignore").decode()
        best_sentences.append(sentence_found)

    return best_sentences #, text_lines_tokens, best_indexes #, news

In [11]:
# do_bm25('quién es el presidente del Perú', dfx, 5)
do_bm25('cuántas vacunas ya llegaron al Perú?', dfx, 5)

amount of contexts:  928


['“Mañana mismo se inicia el proceso con la vacunación del presidente, estamos contentos, nos devuelve la esperanza de enfrentar la crisis sanitaria”, refirió la coordinadora del equipo de vacunación del Ministerio de Salud, Nancy Olivares. Precisó -en diálogo con América Noticias- que junto al mandatario se vacunará en este primer tramo al personal representativo de salud que se encuentra en la primera línea y a los vacunadores. Dijo que se tomó la decisión de que se al mandatario la primera persona en recibir la vacuna para motivar a la población a que también lo haga y demostrar que la vacuna es segura y eficaz. Mientras que la ministra de Salud, Pilar Mazzetti, será una de las últimas en ser vacunadas dentro del personal de salud. “Ha decidido ser una de las ultimas en ser vacunada, decidió que primero sea el personal de salud, nosotros somos administrativos, es un gesto de desprendimiento, vamos a priorizar al personal de mayor vulnerabilidad”, apuntó. Explicó que con las 300,000 